In [1]:
%autosave 300
%reload_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:
import os

os.chdir(
    "/mnt/batch/tasks/shared/LS_root/mounts/clusters/copilot-model-run/code/Users/Soutrik.Chowdhury/pytorch_lightning_journey/lightning_basics"
)
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/copilot-model-run/code/Users/Soutrik.Chowdhury/pytorch_lightning_journey/lightning_basics


In [3]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [4]:
import lightning as L
import torch
import torch.nn.functional as F
from intro.shared_utils import (
    TorchMLP,
    dataset_reader,
    dataset_loader,
    compute_accuracy,
)

ImportError: /anaconda/envs/torch_env/lib/python3.10/site-packages/torch/lib/libtorch_cuda.so: undefined symbol: ncclCommRegister

In [ ]:
torch.manual_seed(71)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


lightning module for model training and validation

In [ ]:
from typing import Any


class LightningMLP(L.LightningModule):

    def __init__(self, model, learning_rate):
        super().__init__()
        self.learning_rate = learning_rate
        self.model = model
        self.criterion = torch.nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        features, true_labels = batch
        logits = self(features)
        loss = self.criterion(logits, true_labels)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        features, true_labels = batch
        logits = self(features)
        loss = self.criterion(logits, true_labels)
        self.log("val_loss", loss, prog_bar=True)

    def test_step(self, batch, batch_idx):
        pass

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(
            self.parameters(), lr=self.learning_rate, momentum=0.9
        )
        return optimizer

In [ ]:
train_data, val_data, test_data = dataset_reader()

In [ ]:
train_dataloader, val_dataloader, test_dataloader = dataset_loader(
    train_data, val_data, test_data, 10
)

In [ ]:
for data, labels in train_dataloader:
    print(data.shape)
    print(len(labels))
    break

for data, labels in val_dataloader:
    print(data.shape)
    print(len(labels))
    break

for data, labels in test_dataloader:
    print(data.shape)
    print(len(labels))
    break

# as we are using basic mlp model flatten() is required

torch.Size([64, 1, 28, 28])
64


torch.Size([64, 1, 28, 28])
64
torch.Size([64, 1, 28, 28])
64


In [ ]:
# basic torch model
torch_model = TorchMLP(input_dim=784, hidden_dim=50, output_dim=10).to(device)

In [ ]:
# lightning model
lightning_model = LightningMLP(model=torch_model, learning_rate=0.01)

In [5]:
trainer = L.Trainer(
    max_epochs=2,
    accelerator="auto",
    devices="auto",
    deterministic=True,
    enable_progress_bar=True,
    enable_model_summary=True,
)

NameError: name 'L' is not defined

In [6]:
trainer.fit(
    model=lightning_model,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

NameError: name 'trainer' is not defined

In [14]:
lightning_model.device

device(type='cpu')

In [15]:
train_acc = compute_accuracy(lightning_model, train_dataloader, device)

cpu
cuda:0


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [ ]:
val_acc = compute_accuracy(lightning_model, val_dataloader, device)

In [ ]:
test_acc = compute_accuracy(lightning_model, test_dataloader, device)

In [ ]:
print(
    f"Train Acc {train_acc*100:.2f}%"
    f" | Val Acc {val_acc*100:.2f}%"
    f" | Test Acc {test_acc*100:.2f}%"
)

In [ ]:
PATH = "models/lightning.pt"
os.path.makedirs(PATH, exists_ok=True)
torch.save(torch_model.state_dict(), PATH)

In [ ]:
# To load model:
# model = PyTorchMLP(num_features=784, num_classes=10)
# model.load_state_dict(torch.load(PATH))
# model.eval()